# API Testing

```cmd
npm install cors
npm install dotenv
touch .env
npm install express
npm install morgan
npm install pg
```
the controller 

routes connect the request into the controller



To test our API we need the module:
```cmd
npm install -D jest supertest
```

In our `package.json`, we add:
```json
"test": "jest --watchAll -- verbose --detectOpenHandles --runInBand",
"coverage": "jest --coverage"
```

Integration test - testing multiple components are working together 

end-to-end - the whole application works, much broader on client-to-client

To start a test we need a test folder and Class file: `Goat.test.js`
```cmd
mkdir __tests__
cd __tests__
mkdir unit
cd unit
mkdir models
cd models
touch Goat.test.js
```

## Testing
### getALL
inside `__test__ > unit > models > Goat.test.js`:
```js
const db = require('../../../database/connect')
const Goat = require('../../../models/Goat')

describe('Goat', () => {

    // Reset the state before running the next test
    beforeEach(() => jest.clearAllMocks()) 
    afterAll(() => jest.resetAllMocks())

    describe('getAll', () =>{
        it('resolves with goats on successful db query', async () => {
            // Arrange
                const mockGoat = [
                    { id: 1, name: 'g1', age: 1},
                    { id: 2, name: 'g2', age: 2},
                    { id: 3, name: 'g3', age: 3}
                ]
                jest.spyOn(db, 'query').mockResolvedValueOnce({rows: mockGoat}) // This intercepts the call and inserts our mock value:
                
            // Act
                const goats = await Goat.getAll()
            // Assert
                expect(goats).toHaveLength(3)
                expect(goats[0]).toHaveProperty('id')
                expect(goats[0].name).toBe('g1')
                expect(db.query).toHaveBeenCalledWith("SELECT * FROM goats")
            
        })
        it('should throw an Error when no goats are found', async () => {
            // Arrange
                jest.spyOn(db, 'query').mockResolvedValueOnce({rows: []})
            // Act & Assert
                await expect(Goat.getAll()).rejects.toThrow('No goats available.')
            
        })
    })
})
```

### findByID
```js
describe('findById', () => {
        it('resolves with goat on successful dbquery' , async () => {
            // Arrange
                const testGoat = [{ id: 1, name: 'goat', age: 22}]
                jest.spyOn(db, 'query').mockResolvedValueOnce({ rows: testGoat})
            // Act
                const result = await Goat.findById(1)
            // Assert
                expect(result).toBeInstanceOf(Goat)
                expect(result.name).toBe('goat')
                expect(result.id).toBe(1)
                expect(db.query).toHaveBeenCalledWith('SELECT * FROM goats WHERE id = $1', [1])
        })
    
        it('should throw an Error when goat is not found', async () =>{
            // Arrange
                jest.spyOn(db, 'query').mockResolvedValueOnce({rows: []})
            // Act & Assert
                await expect(Goat.findById(999)).rejects.toThrow('This goat does not exist!')
        })
    })
```

### create
```js
describe('create', () => {
        it('resolves with goat on successful creation', async () => {
            // Arrange
                const goatData = {name: 'plum', age: 99}
                jest.spyOn(db, 'query').mockResolvedValueOnce({rows: [{...goatData, id: 1}]}) // We use the ... operator to iterate through the variable into the array as this is true to how we would pass data into this create function.
            // Act
                const result = await Goat.create(goatData)
            // Assert
                expect(result).toHaveProperty('id', 1)
                expect(result).toHaveProperty('name', 'plum')
                expect(result).toHaveProperty('age', 99)
                expect(db.query).toHaveBeenCalledWith("INSERT INTO goats(name, age) VALUES ($1, $2) RETURNING *", [goatData.name, goatData.age])
        })
        it('should throw an error when age is missing', async () =>{
            // Arrange
                const incompleteGoatData = {name: 'plum'}
            // Act & Assert
                await expect(Goat.create(incompleteGoatData)).rejects.toThrow('age is missing')
        })
    })
```

## Controller testing
```cmd
cd server/__tests__/unit/
mkdir controllers
cd controllers
touch goats.test.js
```


### setting up the controller test file
```js
// Import the models
const goatsController = require('../../../controllers/goats')
const Goat = require('../../../models/Goat')

// ability to mock functions:
const mockSend      = jest.fn()
const mockJson      = jest.fn()
const mockEnd       = jest.fn()
const mockStatus    = jest.fn(() => ({
    send:   mockSend,
    json:   mockJson,
    end:    mockEnd
}))
const mockRes       = {status: mockStatus}
```

#### reseting variables after running each test
```js
describe('Goats controller', () => {
    beforeEach(() => jest.clearAllMocks())
    afterAll(() => jest.resetAllMocks())
})
```

### index

```js
describe('index', () => {
    it ('should return goats with a status code of 200', async () => {
        // ARRANGE: mock the response from Goat.getAll - it is looking for an array
            const testGoats = ['g1', 'g2']
        // intercept the request 
            jest.spyOn(Goat, "getAll").mockResolvedValue(testGoats)

        // ACT
            await goatsController.index(null, mockRes)

        // ASSERT
            expect(Goat.getAll).toHaveBeenCalledTimes(1)
            expect(mockStatus).toHavebeenCalledWith(200)
            expect(mockSend).toHavebeenCalledWith({data: testGoats})
    })

    it ('should return an error upon failure', async () => {
        // ARRANGE
            jest.spyOn(Goat, 'getAll').mockRejectedValue(new Error('Something happened with the db'))
        // ACT 
            await goatsController.index(null, mockRes)
        // ASSERT
            expect(Goat.getAll).toHaveBeenCalledTimes(1)
            expect(mockStatus).toHavebeenCalledWith(500)
            expect(mockSend).toHavebeenCalledWith({error: 'Something happened with the db'})
    })
})
```

### Show
```js
describe('show', () => {
    let testGoat, mockReq

    beforeEach(() => {
        testGoat = { id: 1, name: 'test goat', age: 22}
        mockReq = {params: {id: 1}}
    })

    if('should return a goat with a 200 status code', async () => {
        // ARRANGE
            jest.spyOn(Goat, 'findById').mockResolvedValue(testGoat)

        // ACT
            await goatsController.show(mockReq, mockRes)

        // ASSERT
            expect(Goat.findById).toHaveBeenCalledTimes(1)
            expect(mockStatus).toHavebeenCalledWith(200)
            expect(mockSend).toHavebeenCalledWith({data: testGoat})
    })

    it('should return an error if goat not found', async () => {
        // ARRRANGE
            jest.spyOn(Goat, 'findById').mockResolvedValue(new Error('Oh no'))

        // ACT
            await goatsController.show(mockReq, mockRes)

        // ASSERT
            expect(Goat.findById).toHaveBeenCalledTimes(1)
            expect(mockStatus).toHavebeenCalledWith(404)
            expect(mockSend).toHavebeenCalledWith({error: 'Oh no'})
    })
})
```

### Create
```js

describe('create', () => {
    it('should create a new goat with status 201 code', async() => {
        // ARRANGE
            let testGoat = {name: 'test goat', age: 2, id: 4}
            const mockReq = {body: testGoat}
            jest.spyON(Goat, 'create').mockResolvedValue(testGoat)

        // ACT
            await goatController.create(mockReq, mockRes)

        // ASSERT
            expect(Goat.create).toHaveBeenCalledTimes(1)
            expect(mockStatus).toHaveBeenCalledWith(201)
            expect(mockSend).toHaveBeenCalledWith({data: testGoat})
    })
    it('should return an error if creation fails', async () => {
        // ARRANGE 
            let testGoat = {name: 'test goat'}
            const mockReq = {body: testGoat}
            jest.spyOn(Goat, 'create').mockRejectedValue(new Error('Oh no'))

        // ACT 
            await goatController.create(mockReq, mockRes)

        // ASSERT
            expect(Goat.create).toHaveBeenCalledTimes(1)
            expect(mockStatus).toHaveBeenCalledWith(200)
            expect(mockSend).toHaveBeenCalledWith({error: 'Oh no'})
    })
})

## self assessment
### Models
#### update
```js
describe('update', () => {
    it('resolves with updated information', async () => {
        // ARRANGE 
            const goatOld = new Goat {name: 'taylor', age: 22, id: 1}
            const goatNew = {name: 'swift', age: 89,}
            jest.spyOn(db, 'query').mockResolvedValueOnce({rows: [{...goatNew, id: 1}]})
        // ACT 
            const result = await goatOld.update(goatNew)
        // ASSERT
            expect(result).toHaveProperty('id', 1)
            expect(result).toHaveProperty('name', 'swift')
            expect(result).toHaveProperty('age', 89)
            expect(db.query).toHaveBeenCalledWith("UPDATE goats SET name = $1, age = $2 WHERE id = $3 RETURNING *", [goatNew.name, goatNew.age, goatOld.id])
    
    })
})

```

## Integrated tests
Setting up our database
WINDOWS
```json
"setup-test-db": "set NODE_ENV=test&& node ./database/setup.js"
```

OSX
```json
"setup-test-db": "NODE_ENV=test node ./database/setup.js"
```

In command line
```cmd
npm run setup-test-db

cd __tests__
mkdir integration
cd integration
touch config.js
```

In the config.js
```js
const {Pool} = require('pg')
const fs = require('fs')
require('dotenv').config()

const resetSQL = fs.readFileSync(__dirname + '/reset.sql').toString()

const resetTestDB = async () => {
    try {
        const db = new Pool ({
            connectionString: process.env.DB_TEST_URL
        })
    await db.query(resetSQL)
    await db.end
    console.log("Test database reset successfully")

    } catch(err) {
        console.error('Could not reset TestDB', err)
        throw err
    }
}

module.exports = {
    resetTestDB
}
```

We want reset the data inside the SQL database with:
```cmd
To create the file

touch reset.sql
```

And to reset the database with sample data:
```sql
TRUNCATE goats RESTART IDENTITY;
-- TRUNCATE deletes all data keeps columns

INSERT INTO goats (name, age)
VALUES
    ('goat 1', 1),
    ('goat 2', 2),
    ('goat 3', 3);
```

In the JSON package
```JSON
"jest": {
    "testPathIgnorePatterns": [
      "./__tests__/integration/config.js"
    ]
  }

```

To create our api tests:
```cmd
touch api.test.js
```

In api.test.js
```js
const request = require('supertest')
const app = require('../../app')
const { resetTestDB } = require('./config')

describe('Goat API Endpoints', () => {
  let api

  beforeEach(async () => {
    await resetTestDB()
  })

  beforeAll(() => {
    api = app.listen(4000, () => {
      console.log('Test server running on port 4000')
    })
  })

  afterAll((done) => {
    console.log('Gracefully closing server')
    api.close(done)
  })
}
```


INDEX
```js
    describe('GET /', () => {
    it('responds to GET / with a message and a description', async () => {
      // ARRANGE
        const response = await request(api).get('/')
      // ACT
        expect(response.statusCode).toBe(200)
      // ASSERT
        expect(response.body.message).toBe('welcome')
        expect(response.body.description).toBe('GOAT API')
    })
  });
```

GET
```js
describe('GET /goats', () => {
    it('should return all goats with a status code 200', async () => {
      // ARRANGE
        const response = await request(api).get('/goats');
      // ACT
        expect(response.status).toBe(200);
      // ASSERT
        expect(response.body.data).toBeInstanceOf(Array);
        expect(response.body.data.length).toBeGreaterThan(0);
    });
  });
```

```js
describe('GET /goats/:id', () => {
    it('should return a specific goat by ID', async () => {
      // ARRANGE
        const goatId = 1;
      // ACT
        const response = await request(api).get(`/goats/${goatId}`);
      // ASSERT
        expect(response.status).toBe(200);
        expect(response.body.data).toHaveProperty('id', goatId);
    });

    it('should return a 404 if goat is not found', async () => {
      // ARRANGE
        const nonExistentGoatId = 999;
      // ACT
        const response = await request(api).get(`/goats/${nonExistentGoatId}`);
      // ASSERT
        expect(response.status).toBe(404);
        expect(response.body.error).toBe('This goat does not exist!');
    });
  });
```

POST route
```js
describe('POST /goats', () => {
    it('should create a new goat ad return it', async () => {
        // ARRANGE 
            const newGoat = {name: 'goaty', age: 22}
        // ACT  
            const response = await request(api).post('/goats').send(newGoat)
        // ASSERT
            expect(response.status).toBe(201)
            expect(response.body.data).toHaveProperty('name', 'goaty')
            expect(response.body.data).toHaveProperty('age', 22)
    })
    it('should return a 400 if required fields are missing', async () => {
        // ARRANGE
            const incompletedGoat = {name: 'ghee'}
        // ACT
            const response = (await request(api).post('/goats')).setEncoding(incompletedGoat)
        // ASSERT
            expect(response.status).toBe(400)
            expect(response.body.error).toBe('age is missing')
    })
})
```

PATCH 
```js
describe('PATCH /goats/:id', () => {
    it('should update an existing goat and return it', async () => {
        // ARRANGE 
            const goatId = 1
            const goatNew = {name: "billy", age: 777}
        // ACT 
            const reponse = await request(api).patch(`/goat/${goatId}`).send(goatNew)
        // ASSERT
            expect(response.status).toBe(200)
            expect(response.body.data).toHaveProperty('name', 'billy')
            expect(response.body.data).toHaveProperty('age', 77)
    })
    it('should return 400 status code for missing data', async () => {
        // ARRANGE
            const goatId = 999
            const goatNew = {name: "billy", age: 777}goatNew 
        // ACT 
            const reponse = await request(api).patch(`/goat/${goatId}`).send(goatNew)
        // ASSERT
            expect(response.status).toBe(400)
            expect(response.body.error).toBe('This goat does not exist!')
    })
})
```